In [1]:
import numpy as np
import re
from itertools import combinations
import math
import time

In [2]:
with open("input_24.txt", "r") as fh:
    content = fh.read().split("\n\n")

initial_bits = {i[0]:int(i[1]) for i in re.findall(r"([xy]\d{2}):\s([01])", content[0])}
logic_rules = {i[3]:(i[1], i[0], i[2]) for i in re.findall(r"([a-z0-9]{3})\s(AND|OR|XOR)\s([a-z0-9]{3})\s->\s([a-z0-9]{3})", content[1])}

In [3]:
# 24a
activations = initial_bits.copy()

def apply_logic(operator, operand1, operand2, activations):
    if operand1 not in activations or operand2 not in activations:
        return None
    if operator == "AND":
        return activations[operand1] and activations[operand2]
    if operator == "OR":
        return activations[operand1] or activations[operand2]
    if operator == "XOR":
        return activations[operand1] ^ activations[operand2]
    print("Error: code should not reach here.")

def evaluate_system(activations, logic_rules):
    # Evaluate what can be evaluated, and keep track of what could not be evaluated yet (missing operands).
    # Keep repeating until there are no more missing operands
    delayed_calculation = logic_rules.copy()
    counter = 0 # Needed to avoid infinite while loops in part b
    while len(delayed_calculation) and counter < 200:
        counter += 1
        new_delayed_calculation = []
        for dc in delayed_calculation:
            operator, operand1, operand2 = logic_rules[dc]
            res = apply_logic(operator, operand1, operand2, activations)
            if res is None:
                new_delayed_calculation.append(dc)
            else:
                activations[dc] = res
        delayed_calculation = new_delayed_calculation.copy()
    
    # Compute decimal from bits (z00 being least significant bit)
    exp_counter = 0
    result = 0
    for k in sorted(activations):
        if k[0] == "z":
            result += activations[k] * 2**exp_counter
            exp_counter += 1
    return result
    
print("24a", evaluate_system(activations, logic_rules))

24a 61886126253040


In [4]:
# 24b
activations = initial_bits.copy()
activations

def sum_system(x, y, logic_rules):
    activations = initial_bits.copy()
    num_bits = sum([1 for k in initial_bits.keys() if k[0] == "x"])
    for e in range(num_bits):
        activations[f"x{e:02}"] = x % 2
        activations[f"y{e:02}"] = y % 2
        x >>= 1
        y >>= 1
    return evaluate_system(activations, logic_rules)

def get_related_outputs(gate, logic_rules):
    '''Get all outputs that are part of getting to a certain z-gate'''
    k = f"z{gate:02}"
    related_outputs = [k]
    k2s = logic_rules[k][1:]
    while len(k2s):
        new_k2s = []
        for k2 in k2s:
            if k2[0] not in ["x", "y"]:
                related_outputs.append(k2)
                new_k2s += logic_rules[k2][1:]
        k2s = new_k2s.copy()
    return related_outputs

def swap(logic_rules, k1, k2):
    '''Swap two elements in logic_rules.'''
    logic_rules_output = logic_rules.copy()
    k_temp = logic_rules[k1]
    logic_rules_output[k1] = logic_rules[k2]
    logic_rules_output[k2] = k_temp
    return logic_rules_output

def test_logic(gate, logic_rules):
    '''Compare the sum generated by the system and the actual sum.'''
    xy_values = [0, 2**gate, 2**(gate-1), 2**gate + 2**(gate-1)] if gate > 0 else [0, 1]
    gate_correct = True
    for x in xy_values:
        for y in xy_values:
            z_actual = bin(x + y)[2:].zfill(num_gates)
            z = bin(sum_system(x, y, logic_rules))[2:].zfill(num_gates)
            if z_actual[::-1][gate] != z[::-1][gate]:
                gate_correct = False
                break
        if not gate_correct: break
    return gate_correct

t0 = time.time()
potential_swap_elements = list(logic_rules.keys())
num_gates = len([k for k in logic_rules if k[0] == "z"])
adj_logic_rules = logic_rules.copy()
swaps = []

for gate in range(num_gates-1):
    # Check for all combinations of this gate, and the carry (bit to the right)
    xy_values = [0, 2**gate, 2**(gate-1), 2**gate + 2**(gate-1)] if gate > 0 else [0, 1]
    gate_correct = test_logic(gate, adj_logic_rules)

    # Either indicate that certain outputs are not swap candidates, or start swapping
    gate_outputs = get_related_outputs(gate, adj_logic_rules)
    if gate_correct:
        potential_swap_elements = [e for e in potential_swap_elements if e not in gate_outputs]
        print(f"Gate z{gate:02} is correct.")
    else:
        for o in gate_outputs:
            for s in potential_swap_elements:
                # Z-values should not be swapped
                if o == s or (o[0] == "z" and s[0] == "z"): continue
                temp_logic_rules = swap(adj_logic_rules, o, s)
                gate_correct = test_logic(gate, temp_logic_rules)
                if gate_correct:
                    # This swap solved the system
                    print(f"Gate z{gate:02}: swap elements {o} and {s}.")
                    swaps.append((o, s))
                    adj_logic_rules = temp_logic_rules.copy()
                    break
            if gate_correct: break
        if not gate_correct:
            print(f"Gate z{gate:02}: no swap found.")

# List swaps found
all_swaps = [s[0] for s in swaps] + [s[1] for s in swaps]
print()
print("24b:", ",".join(sorted(all_swaps)), f"in {time.time()-t0} sec.")

Gate z00 is correct.
Gate z01 is correct.
Gate z02 is correct.
Gate z03 is correct.
Gate z04 is correct.
Gate z05 is correct.
Gate z06 is correct.
Gate z07: swap elements z07 and nqk.
Gate z08 is correct.
Gate z09 is correct.
Gate z10 is correct.
Gate z11 is correct.
Gate z12 is correct.
Gate z13 is correct.
Gate z14 is correct.
Gate z15 is correct.
Gate z16 is correct.
Gate z17: swap elements fgt and pcp.
Gate z18 is correct.
Gate z19 is correct.
Gate z20 is correct.
Gate z21 is correct.
Gate z22 is correct.
Gate z23 is correct.
Gate z24: swap elements z24 and fpq.
Gate z25 is correct.
Gate z26 is correct.
Gate z27 is correct.
Gate z28 is correct.
Gate z29 is correct.
Gate z30 is correct.
Gate z31 is correct.
Gate z32: swap elements z32 and srn.
Gate z33 is correct.
Gate z34 is correct.
Gate z35 is correct.
Gate z36 is correct.
Gate z37 is correct.
Gate z38 is correct.
Gate z39 is correct.
Gate z40 is correct.
Gate z41 is correct.
Gate z42 is correct.
Gate z43 is correct.
Gate z44 is 

In [5]:
# List all gates related to a certain output; used for solving 24b
for k in sorted(logic_rules):
    if k[0] == "z":
        print(f"{k} = {logic_rules[k][1]} {logic_rules[k][0]} {logic_rules[k][2]}")
        k2s = logic_rules[k][1:]
        while len(k2s):
            new_k2s = []
            for k2 in k2s:
                if k2[0] not in ["x", "y"]:
                    print(f"{k2} = {logic_rules[k2][1]} {logic_rules[k2][0]} {logic_rules[k2][2]}")
                    new_k2s += logic_rules[k2][1:]
            k2s = new_k2s.copy()
        print()

z00 = y00 XOR x00

z01 = hfm XOR hqt
hfm = y00 AND x00
hqt = y01 XOR x01

z02 = dmw XOR bfr
dmw = qng OR hkm
bfr = y02 XOR x02
qng = x01 AND y01
hkm = hqt AND hfm
hqt = y01 XOR x01
hfm = y00 AND x00

z03 = wmn XOR rmb
wmn = x03 XOR y03
rmb = fvk OR dpq
fvk = bfr AND dmw
dpq = y02 AND x02
bfr = y02 XOR x02
dmw = qng OR hkm
qng = x01 AND y01
hkm = hqt AND hfm
hqt = y01 XOR x01
hfm = y00 AND x00

z04 = qhk XOR fcw
qhk = y04 XOR x04
fcw = jnw OR rjn
jnw = wmn AND rmb
rjn = y03 AND x03
wmn = x03 XOR y03
rmb = fvk OR dpq
fvk = bfr AND dmw
dpq = y02 AND x02
bfr = y02 XOR x02
dmw = qng OR hkm
qng = x01 AND y01
hkm = hqt AND hfm
hqt = y01 XOR x01
hfm = y00 AND x00

z05 = vmd XOR wbn
vmd = y05 XOR x05
wbn = nsk OR kjg
nsk = qhk AND fcw
kjg = y04 AND x04
qhk = y04 XOR x04
fcw = jnw OR rjn
jnw = wmn AND rmb
rjn = y03 AND x03
wmn = x03 XOR y03
rmb = fvk OR dpq
fvk = bfr AND dmw
dpq = y02 AND x02
bfr = y02 XOR x02
dmw = qng OR hkm
qng = x01 AND y01
hkm = hqt AND hfm
hqt = y01 XOR x01
hfm = y00 AND x